# Pression statique - Mesures

In [ ]:
from pduino import *
liste_ports()

## Etalonnage  du capteur

In [1]:
# --- Etalonnage du capteur - Pression statique
from pduino import *
from juduino import *

# modifiez le port en fonction de votre configuration
port = 'COM4'

my_interface = Interface(port, [('P', 'hPa'), ('h', 'm')], 'points')

def acquisition():
    # exécuté lorsque l'utilisateur clique sur le bouton 'Mesure'
    P = calcule_pression(float(my_interface.set_command('mesure')))
    print('P = {:.2f} hPa'.format(P))

def calcule_pression(b):
    k = float(valeur_k.value)
    uref = 5.0;
    uref_CAN = 1.1
    u = b / 1023 * uref_CAN;
    # valeur à modifier éventuellement
    p = (u-k)*7000 / (uref-k);
    p = round(p, 2)
    return p

# --- programme principal
valeur_k = my_interface.add_saisie('Valeur de k', '0.114')
my_interface.add_bouton('Mesure', acquisition)
my_interface.affiche()

## Mesures des points couples de points (h, P)

In [3]:
# --- mesures ponctuelles - Pression statique
import threading
import ipywidgets as widgets
import time

my_int_ponctuel = Interface(port, [('P', 'hPa'), ('h', 'm')], 'points')    
thread = None
arreter_thread = False

def mon_thread():
    car = '...'
    while not arreter_thread:
        data = my_int_ponctuel.arduino.arduino.readline()
        if data != '':
            mesure = data.decode()[:-2]
            try:
                P = calcule_pression(float(mesure))
                texte_pression.value = str(P)
                if X.value == car:
                    X.value = ''
                else:
                    X.value = car
            except:
                pass

def disable_boutons(B):
    if B:
        bouton_temps_reel.button_style = 'info'
    else:
        bouton_temps_reel.button_style = ''
        X.value = ''
        
def temps_reel():
    global thread, arreter_thread
    if thread is None:
        while my_int_ponctuel.arduino.arduino.readline().decode() != '' : {}
        thread = threading.Thread(target=mon_thread)
        arreter_thread = False 
        thread.start()
        my_int_ponctuel.arduino.arduino.write('{}'.format('mesure_regulier').encode())
        disable_boutons(True)
    else:
        arreter_thread = True
        thread.join()
        my_int_ponctuel.arduino.arduino.write('{}'.format('stop').encode())
        arreter_thread = False
        thread = None
        disable_boutons(False)
    
def acquisition_pression():
    # exécuté lorsque l'utilisateur clique sur le bouton 'Mesure'
    h = float(saisie_h.value)
    P = float(texte_pression.value)
    print('h = {:.1f} cm ; P = {:.2f} hPa'.format(h, P))
    my_int_ponctuel.ajoute_point(h, P)
    
def supp_dernier_point():
    # exécuté lorsque l'utilisateur clique sur le bouton 'Supp dernier point'
    if my_int_ponctuel.supprime_dernier_point():
        print("Dernier point supprimé")
    else:
        print('Pas de dernier point')
    
def tableaux():
    # exécuté lorsque l'utilisateur clique sur le bouton 'Tableaux'
    H, P = my_int_ponctuel.get_valeurs()
    n = 40
    if len(H) != 0:
        print(liste_valeurs('H', H, n))
        print(liste_valeurs('P', P, n))     
    else:
        print('Pas de mesures')
    
# --- programme principal
texte_pression, X = widgets.Text(value='', description='P (hPa)'), widgets.Label(value='')
display(widgets.HBox([texte_pression, X]))
saisie_h = my_int_ponctuel.add_saisie('h (cm)', '0')
bouton_temps_reel = my_int_ponctuel.add_bouton('Mesures', temps_reel)
bouton_mesure = my_int_ponctuel.add_bouton('Ajout point', acquisition_pression)
bouton_supp = my_int_ponctuel.add_bouton('Supp. dernier point', supp_dernier_point)
bouton_tableau = my_int_ponctuel.add_bouton('Tableaux', tableaux)
my_int_ponctuel.affiche()